In [25]:
from bs4 import BeautifulSoup
from urllib import request
from tqdm import tqdm
import json, pandas as pd
from io import StringIO
import pickle

---

In [26]:
def f(n):
    url = 'http://www.10000recipe.com/recipe/' + str(n)
    req = request.Request(url)
    code = request.urlopen(url).read()
    soup = BeautifulSoup(code, 'html.parser')

    answer = {}
    answer['이름'] = soup.find('div', 'view2_summary').find('h3').get_text()
    answer['썸네일'] = soup.find('div', 'view2_pic').find('img', id='main_thumbs')['src']
    answer['정보'] = soup.find('div', 'view2_summary_info').get_text().replace('\n', '')
    res = soup.find('div', 'ready_ingre3')
    try:
        ingre = []
        for x in res.find_all('ul'):
            for y in x.find_all('li'):
              temp = y.get_text().replace('\n', '').replace('구매', '').split(' ')
              if temp[-1]:    # 정량이 없을 수 있다
                ingre.append([temp[0], temp[-1]])
              else:
                ingre.append([temp[0]])

        answer['재료'] = [ingre]
    except (AttributeError):
        return

    res = soup.find('div', 'view_step')
    i = 0
    step = []
    step_img = []
    for n in res.find_all('div', 'view_step_cont'):
        i += 1
        try:
            step_img.append(soup.find('div', id='stepimg' + str(i)).img['src'])
        except:
            pass
        step.append('#' + str(i) + ' ' + n.get_text().replace('\n', ''))
    if not step:
      return False
    answer['순서'] = [step]
    answer['순서img'] = [step_img]    # 없을수도...?
    return answer

In [36]:
with open("final.pkl", "rb") as f:
    x = pickle.load(f)
f.close()

In [38]:
df = pd.DataFrame()
for i in tqdm(x):
    a = f(i)
    if not a:    # 순서 표기가 없어서 false 로 반환 되는 것이 있음
        continue
    a = json.dumps(a, ensure_ascii=False)
    temp = pd.read_json(StringIO(a))
    if temp.shape[1] == 6:
        df = pd.concat([df, temp], ignore_index=True)

  0%|                                                                                       | 0/159809 [00:00<?, ?it/s]


TypeError: '_io.BufferedReader' object is not callable

In [ ]:
df.to_csv('a.csv', index=None, encoding="utf-8-sig")